#### 한국어 뉴스 헤드라인을 이용하여, 뉴스의 주제를 분류
###### 평가산식 : Accuracy
Public 평가 : 테스트 데이터 중 랜덤 샘플 된 50%로 채점, 대회 기간 중 공개

Private 평가 : 나머지 50 % 테스트 데이터로 채점, 대회 종료 직후 공개

#### 데이터 변수 설명

In [1]:
# index : 헤드라인 인덱스
# title : 뉴스 헤드라인
# topic_idx : 뉴스 토픽 인덱스 값
# topic : 실제 뉴스 토픽

#### 예측해야 할 칼럼 : topic_idx 

In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
topic = pd.read_csv("topic_dict.csv")
display(train,test,topic)

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4
...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1
45652,45652,답변하는 배기동 국립중앙박물관장,2


,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다
3,45657,김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
4,45658,회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계


,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


In [4]:
all_data = pd.concat([train,test])

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4.0
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4.0
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4.0
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4.0
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4.0
...,...,...,...
9126,54780,인천 오후 3시35분 대설주의보…눈 3.1cm 쌓여,NaN
9127,54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합,NaN
9128,54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개,NaN
9129,54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계,NaN


In [5]:
text = list(all_data["title"])
text[:5]

['인천→핀란드 항공기 결항…휴가철 여행객 분통',
 '실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화',
 '이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것',
 'NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합',
 '시진핑 트럼프에 중미 무역협상 조속 타결 희망']

In [6]:
# 한글 형태소 분석기 중 꼬꼬마를 사용하였다.
from tqdm import tqdm
from konlpy.tag import Kkma
kkma = Kkma()
kkma_list = []
for i in tqdm(text):
    k = kkma.morphs(i)
    kkma_list.append(k)

100%|██████████| 54785/54785 [15:30<00:00, 58.88it/s]


In [7]:
import warnings
warnings.filterwarnings(action = "ignore")
kkma_list = np.array(kkma_list)
kkma_list

array([list(['인천', '→', '핀란드', '항공기', '결항', '…', '휴가', '철', '여행객', '분통']),
       list(['실리콘', '밸리', '넘어서', '겠', '다', '…', '구', '글', '15', '조', '원', '들이', '어', '美', '전역', '거점', '화']),
       list(['이', '란', '외무', '긴장', '완화', '해결책', '은', '미국', '이', '경제', '전쟁', '멈추', '는', '것']),
       ...,
       list(['40', '년', '전', '부마', '항쟁', '부산', '시위', '사진', '2', '점', '최초', '공개']),
       list(['게시판', '아리랑', 'TV', '아프리카', '개발', '은행', '총회', '개회식', '생중', '계']),
       list(['유영', '민', '과기', '장관', '강', '소', '특구', '는', '지역', '혁신', '의', '중심', '…', '지원', '책', '강구'])],
      dtype=object)

In [8]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(kkma_list)

In [9]:
text = tk.texts_to_sequences(kkma_list) 
text[0]

[460, 716, 5654, 2864, 5160, 1, 1879, 498, 3420, 8897]

In [10]:
# 데이터 차원 맞춰주기
from keras.preprocessing.sequence import pad_sequences
pad_text = pad_sequences(text)
pad_text

array([[   0,    0,    0, ...,  498, 3420, 8897],
       [   0,    0,    0, ..., 1956, 3337,  244],
       [   0,    0,    0, ..., 1880,    6,   67],
       ...,
       [   0,    0,    0, ...,  190,  726,  158],
       [   0,    0,    0, ..., 5850, 3017, 1032],
       [   0,    0,    0, ...,  114,  730, 5360]], dtype=int32)

In [11]:
# 하나의 데이터에는 26개의 단어가 들어가 있다.
pad_text.shape

(54785, 26)

In [12]:
train_text = pad_text[:len(train)]
test_text = pad_text[len(train):]

In [13]:
# 단어 전체 종류의 개수
len(tk.word_index)

27109

In [14]:
result_text = 0

from sklearn.model_selection import StratifiedKFold
from keras import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import Adam

skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 82)  
for train_index, valid_index in skf.split(train_text,train["topic_idx"]):
    es = EarlyStopping(patience = 15, verbose = 0) 
    mc = ModelCheckpoint("best.h5",save_best_only = True, verbose = 0)
   
    X_train, X_valid = train_text[train_index], train_text[valid_index] 
    y_train, y_valid = train["topic_idx"].iloc[train_index], train["topic_idx"].iloc[valid_index] 
    
    model = Sequential()
    model.add(Embedding(27110,15, input_length = 26)) 
    model.add(GlobalAveragePooling1D())
    model.add(Dense(7,activation = "softmax"))
   
    model.compile(metrics=["acc"], optimizer = Adam(lr = 0.0005), loss = "sparse_categorical_crossentropy")
    history = model.fit(X_train, y_train, validation_data = (X_valid,y_valid), epochs = 500, callbacks = [es,mc],verbose = 0)
    model.load_weights("best.h5")
    result_text += model.predict(test_text) / 5    

In [15]:
sub = pd.read_csv("sample_submission.csv")
sub["topic_idx"] = result_text.argmax(1)
sub

,index,topic_idx
0,45654,3
1,45655,3
2,45656,2
3,45657,2
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,2
9129,54783,0


In [16]:
sub.to_csv("sub_konlpy.csv",index=False)

#### (7월 9일 기준) 참가자수 : 209명, 제출자수 : 63명, 등수 : 21등 점수 : 0.8276